# Import Libraries

In [1]:
#standard imports
import numpy as np
import pandas as pd
#sql imports
from sqlalchemy import create_engine 


# Title Basics Normalization

In [2]:
#importing data from previous
data = 'Data/title_basics.csv.gz'
basics = pd.read_csv(data)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [3]:
#creating list of genre from genres column
basics['genre_split'] = basics['genres'].str.split(',')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,[Drama]
3,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War","[Drama, War]"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
...,...,...,...,...,...,...,...,...,...,...
136567,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama,[Drama]
136568,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"
136569,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020.0,NaN,84,Thriller,[Thriller]
136570,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History","[Drama, History]"


In [4]:
#exploding genres to seperate into new rows
exploded_genres=basics.explode('genre_split')
#saving all unique genres
unique_genres=sorted(exploded_genres['genre_split'].unique())

In [5]:
#creating Title Genres Table 
title_genres = exploded_genres[['tconst', 'genre_split']].copy()
title_genres.head()

,tconst,genre_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama


In [6]:
## Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

In [7]:
#make new gnre id and remove old column
title_genres['genre_id'] = title_genres['genre_split'].map(genre_map)
title_genres = title_genres.drop(columns='genre_split')

In [8]:
#inspecting for success
title_genres.head()

,tconst,genre_id
0,tt0035423,5
0,tt0035423,9
0,tt0035423,18
1,tt0062336,7
2,tt0069049,7


In [10]:
#creating genres table 
genres = pd.DataFrame({'genre_name':genre_map.keys(),
                       'genre_id':genre_map.values()})
genres.head()

,genre_name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


In [11]:
#removing unneccecary columns from basics
basics = basics.drop(columns=['originalTitle', 'isAdult', 'titleType', 'genres', 'genre_split'])
basics.head()

,tconst,primaryTitle,startYear,endYear,runtimeMinutes
0,tt0035423,Kate & Leopold,2001.0,NaN,118
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020.0,NaN,70
2,tt0069049,The Other Side of the Wind,2018.0,NaN,122
3,tt0079644,November 1828,2001.0,NaN,140
4,tt0088751,The Naked Monster,2005.0,NaN,100


# Ratings Normalization

In [13]:
#import data
data = 'Data/title_ratings.csv.gz'
ratings = pd.read_csv(data)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1924
1,tt0000002,5.8,259
2,tt0000003,6.5,1737
3,tt0000004,5.6,174
4,tt0000005,6.2,2550


already all set

# TMDB normilization

In [14]:
#import data 
data = 'DATA/final_results_movie_00-01.csv.gz'
tmdb = pd.read_csv(data)
tmdb.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.3,1158.0,PG-13
2,tt0079644,0.0,/79axmuH1UGkB7m72jjB9rPff9om.jpg,NaN,0.0,"[{'id': 10752, 'name': 'War'}]",NaN,285529.0,id,November 1828,...,0.0,140.0,"[{'english_name': 'Indonesian', 'iso_639_1': '...",Released,NaN,November 1828,0.0,0.0,0.0,NaN
3,tt0089067,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,210258.0,es,El día de los albañiles 2,...,0.0,90.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,The laborers are back full of love and laughs.,El día de los albañiles 2,0.0,7.2,71.0,NaN
4,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.0,3.0,NaN


In [15]:
#deleting unneccesary columns
tmdb = tmdb[['id', 'revenue', 'budget', 'certification']]
tmdb.head()

,id,revenue,budget,certification
0,NaN,NaN,NaN,NaN
1,11232.0,76019048.0,48000000.0,PG-13
2,285529.0,0.0,0.0,NaN
3,210258.0,0.0,0.0,NaN
4,151007.0,0.0,0.0,NaN


# Creating mySQL database